In [1]:
CREATE TABLE recycling.Businesses (
    BusinessID INT IDENTITY(1,1) PRIMARY KEY,
    Name NVARCHAR(255) NOT NULL,
    FormattedAddress NVARCHAR(500) NOT NULL,
    Latitude DECIMAL(10, 8),
    Longitude DECIMAL(11, 8),
    PhoneNumber NVARCHAR(50),
    Website NVARCHAR(500),
    Rating DECIMAL(3, 2),
    PlaceID NVARCHAR(255),  -- Google Places ID
    DateAdded DATETIME2 DEFAULT SYSUTCDATETIME(),
    LastUpdated DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
);

-- Create indexes for better search performance
CREATE INDEX IX_Businesses_Location ON recycling.Businesses(Latitude, Longitude);
CREATE INDEX IX_Businesses_Name ON recycling.Businesses(Name); 

Commands completed successfully.

Total execution time: 00:00:00.034

In [2]:
CREATE TABLE recycling.BusinessServices (
    ServiceID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    ServiceName NVARCHAR(100) NOT NULL,
    Description NVARCHAR(500),
    IsBookingEnabled BIT DEFAULT 0,
    PriceInfo NVARCHAR(MAX), -- JSON field for flexible pricing structure
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME()
); 

Commands completed successfully.

Total execution time: 00:00:00.053

In [3]:
CREATE TABLE recycling.BusinessHours (
    HoursID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    DayOfWeek TINYINT,  -- 0 = Sunday, 1 = Monday, etc.
    OpenTime TIME,
    CloseTime TIME,
    IsClosed BIT DEFAULT 0
);

Commands completed successfully.

Total execution time: 00:00:00.027

In [4]:
CREATE TABLE recycling.MaterialCategories (
    CategoryID INT IDENTITY(1,1) PRIMARY KEY,
    CategoryName NVARCHAR(100) NOT NULL,
    Description NVARCHAR(500),
    CONSTRAINT UQ_CategoryName UNIQUE (CategoryName)
); 

Commands completed successfully.

Total execution time: 00:00:00.044

In [5]:
CREATE TABLE recycling.BusinessMaterials (
    BusinessID INT,
    CategoryID INT,
    IsVerified BIT DEFAULT 0,
    VerificationSource NVARCHAR(50),
    DateVerified DATETIME2,
    PRIMARY KEY (BusinessID, CategoryID),
    FOREIGN KEY (BusinessID) REFERENCES recycling.Businesses(BusinessID),
    FOREIGN KEY (CategoryID) REFERENCES recycling.MaterialCategories(CategoryID)
);

CREATE INDEX IX_BusinessMaterials_Category ON recycling.BusinessMaterials(CategoryID); 

Commands completed successfully.

Total execution time: 00:00:00.052

In [6]:
CREATE TABLE recycling.BusinessAccounts (
    AccountID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT UNIQUE FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    Email NVARCHAR(255) NOT NULL UNIQUE,
    IsEmailVerified BIT DEFAULT 0,
    VerificationDate DATETIME2,
    LastLoginDate DATETIME2,
    AccountStatus NVARCHAR(20) DEFAULT 'PENDING', -- PENDING, ACTIVE, SUSPENDED
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME()
); 

Commands completed successfully.

Total execution time: 00:00:00.045

In [7]:
CREATE TABLE recycling.Users (
    UserID INT IDENTITY(1,1) PRIMARY KEY,
    Email NVARCHAR(255) NOT NULL UNIQUE,
    PhoneNumber NVARCHAR(50),
    FirstName NVARCHAR(100),
    LastName NVARCHAR(100),
    DefaultAddress NVARCHAR(500),
    DefaultLatitude DECIMAL(10, 8),
    DefaultLongitude DECIMAL(11, 8),
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
); 

Commands completed successfully.

Total execution time: 00:00:00.044

In [8]:
CREATE TABLE recycling.UserSubscriptions (
    SubscriptionID INT IDENTITY(1,1) PRIMARY KEY,
    UserID INT FOREIGN KEY REFERENCES recycling.Users(UserID),
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    NotificationsEnabled BIT DEFAULT 1,
    SubscriptionDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL,
    UNIQUE (UserID, BusinessID)
); 

Commands completed successfully.

Total execution time: 00:00:00.032

In [9]:
CREATE TABLE recycling.CollectionBookings (
    BookingID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    UserID INT FOREIGN KEY REFERENCES recycling.Users(UserID),
    BookingType NVARCHAR(20), -- 'PICKUP', 'DROPOFF'
    ScheduledDate DATE,
    ScheduledTimeSlot NVARCHAR(50),
    Status NVARCHAR(20), -- 'PENDING', 'CONFIRMED', 'COMPLETED', 'CANCELLED'
    Address NVARCHAR(500),
    Latitude DECIMAL(10, 8),
    Longitude DECIMAL(11, 8),
    Notes NVARCHAR(500),
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
);

Commands completed successfully.

Total execution time: 00:00:00.029

In [10]:
CREATE TABLE recycling.WasteRecords (
    RecordID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    MaterialCategoryID INT FOREIGN KEY REFERENCES recycling.MaterialCategories(CategoryID),
    Quantity DECIMAL(10,2),
    Unit NVARCHAR(20),
    RecordDate DATE,
    Notes NVARCHAR(500),
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME()
);

Commands completed successfully.

Total execution time: 00:00:00.043

In [14]:
CREATE TABLE recycling.MaterialTransaction (
    TransactionID INT IDENTITY(1,1) PRIMARY KEY,
    RecordID INT FOREIGN KEY REFERENCES recycling.WasteRecords(RecordID),
    CategoryID INT FOREIGN KEY REFERENCES recycling.MaterialCategories(CategoryID),
    Quantity DECIMAL(10,2),
    Unit NVARCHAR(20),
    PricePerUnit DECIMAL(10,2),
    TotalPrice DECIMAL(10,2),
    Notes NVARCHAR(500),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
); 

Commands completed successfully.

Total execution time: 00:00:00.032